### INDEX :
0. Model Training Things
1. web scrapping for preparing datasets
2. Vocabulary in nlp
3. Data preprocessing steps in NLP
4. REGEX
5. Filtering and Transformatino of text
6. Tokenization and Structuring of text
7. Analysis and representation of text
8. Embedding TEchniques
    frequency based
    prediction based
9. comparing two strings
10. performance metrics
11. computatinal and context window length-kv chache things

In Large Language Models (LLMs), **Key-Value (KV) Caching** is a technique used to store intermediate computations, particularly in the attention mechanism of Transformers. It helps improve both inference efficiency and memory usage.



## **Key-Value Cache in LLMs**
During training and inference, Transformer models process input sequences using **self-attention**. In self-attention, every token attends to all previous tokens, requiring repeated computation for the same tokens in autoregressive decoding. KV caching optimizes this process.

### **How It Works**
- In **causal attention (used in autoregressive models like GPT)**, each new token requires attending to all previous tokens.
- **Without KV caching**: Every time a new token is generated, the model recomputes the attention for all past tokens, leading to redundant computations.
- **With KV caching**: The model **stores** the computed key-value pairs for past tokens in a cache, allowing it to reuse them when processing new tokens. This prevents recomputation and speeds up inference.

Yes, the **context length (token window size)** in LLMs is directly influenced by the **KV cache size**, but it's not the only factor. Let’s break it down:



## **How KV Cache Affects Context Length**
1. **KV Cache Stores Past Tokens' Key-Value Pairs**  
   - In autoregressive models, attention is applied to all previous tokens.
   - The KV cache saves these **key-value pairs** for fast lookup instead of recomputing them.
   - The size of this cache determines how many past tokens the model can efficiently "remember."

2. **Longer Context Requires Larger KV Cache**  
   - If a model supports a **4K token context**, it must store the KV pairs for all **4,096 tokens**.
   - A **longer context (e.g., 32K tokens like GPT-4 Turbo)** means the KV cache needs to store more data.
   - Since each token's key-value pairs require memory, larger contexts need **more VRAM or RAM**.

3. **Trade-offs: Memory vs. Performance**  
   - **Memory Constraint**: Storing KV pairs for long sequences increases memory usage, especially for large models.
   - **Inference Speed**: Longer contexts slow down inference because the attention computation scales as **O(N²)** (for standard attention).
   - **Optimization Techniques**:
     - **Sliding Window Attention**: Instead of storing all past tokens, models like Mistral use a rolling window of attention.
     - **Grouped Query Attention (GQA)**: Used in LLaMA 3 to optimize memory usage.
     - **Memory-efficient attention**: FlashAttention speeds up computations for large contexts.



## **Other Factors Affecting Context Length**
While KV cache is crucial, context length also depends on:
- **Model Architecture**: Some architectures (e.g., Transformers with Rotary Positional Embeddings) handle long contexts better.
- **Positional Encoding**: Models with **ALiBi, RoPE, or Attention Sink** can extend context length.
- **Training Data**: If a model is never trained on long sequences, increasing KV cache alone won’t help.

The FlashAttention paper introduces a memory-efficient attention mechanism that significantly speeds up Transformer models while reducing memory usage. Below are the key papers for both FlashAttention 1 and FlashAttention 2:

### **1️⃣ FlashAttention 1 Paper (2022)**
**Title:** *FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness*  
**Authors:** Tri Dao, Daniel Y. Fu, Stefano Ermon, Atri Rudra, Christopher Ré  
**Paper Link:** [https://arxiv.org/abs/2205.14135](https://arxiv.org/abs/2205.14135)  

#### **Key Contributions**
- **Memory-efficient self-attention:** Reduces GPU memory usage by 50-80% while maintaining exact attention computation.
- **Optimized GPU memory access:** Uses tiling and recomputation strategies to avoid redundant memory loads.
- **2-4x speedup over standard attention** without loss of accuracy.
- **Enabled longer context lengths** in models like GPT-4 and Gemini.

### **2️⃣ FlashAttention 2 Paper (2023)**
**Title:** *FlashAttention-2: Faster Attention with Better Parallelism and Work Partitioning*  
**Authors:** Tri Dao  
**Paper Link:** [https://arxiv.org/abs/2307.08691](https://arxiv.org/abs/2307.08691)  

#### **Key Improvements Over FlashAttention 1**
- **Pipelined computation:** Overlaps memory operations with computation for better efficiency.
- **Better work partitioning:** Reduces synchronization overhead for multi-head attention.
- **Handles variable-length sequences efficiently**, making it even faster.
- **Achieves 2x speedup over FlashAttention 1.0.**



---
---

The **time complexity** of ChatGPT (and Transformer-based models in general) depends on the **self-attention mechanism** and its optimizations.  


## **1. Time Complexity of Standard Transformer (GPT)**
For a **decoder-only Transformer** like GPT:  

### **Self-Attention Complexity**  
- **Per-token computation:** \( O(N) \)  
- **Per-layer computation for full sequence:** \( O(N^2 d) \)  
  - \( N \) = number of tokens (context length)  
  - \( d \) = embedding dimension  

### **Feedforward Network Complexity**  
- \( O(N d^2) \) (since each token passes through fully connected layers)  

### **Total Complexity Per Layer**  
For a model with **L layers**, the total complexity is:  
\[
O(L (N^2 d + N d^2))
\]  
- **Quadratic term \( O(N^2) \)** is the bottleneck, limiting long contexts.  
- **For large \( d \), \( O(N d^2) \) can also be expensive.**  


## **2. Complexity During Inference (Autoregressive Generation)**
During **autoregressive text generation**, only the newly generated token attends to previous tokens, leading to:  

- **Without KV Cache**: \( O(N^2 d) \) per step (due to recomputing attention for all tokens).  
- **With KV Cache**: \( O(N d) \) per step (each new token only attends to stored KV pairs).  

Thus, KV caching **reduces inference complexity from \( O(N^2) \) to \( O(N) \) per step**, making real-time generation feasible.



## **3. Optimized GPT Models (ChatGPT, GPT-4, etc.)**
To handle **longer contexts (e.g., 8K, 32K tokens)**, models like GPT-4 use optimizations:  

### **1️⃣ FlashAttention (Speeds up Attention)**
- Reduces memory usage and speeds up softmax computation.
- Lowers **practical** complexity to nearly **\( O(N) \)** per token.  

### **2️⃣ Grouped Query Attention (GQA)**
- Instead of each query attending to all keys, groups of queries share keys.
- Reduces the number of computations from **\( O(N^2) \) to a more efficient form**.

### **3️⃣ Mixture of Experts (MoE) in GPT-4 Turbo**
- Uses only a **subset of parameters per token**.
- Reduces effective complexity compared to dense models.



## **Final Complexity Estimates**
| Model Stage | Complexity |
|-------------|------------|
| **Training (full self-attention)** | \( O(L N^2 d + L N d^2) \) |
| **Inference (with KV cache)** | \( O(L N d) \) per token |
| **Optimized GPT Models (FlashAttention, GQA, MoE)** | **Closer to \( O(N) \) per token** |

**Conclusion:**  
- **Training** is **quadratic** in sequence length $ O(N^2) $.  
- **Inference (with KV caching)** is **linear $ O(N) $ per token**.  
- **Optimizations like FlashAttention and GQA** reduce effective complexity.

---
---


### model training things

## **1️⃣ Multi-Token Prediction**  
### **What is it?**  
- In standard autoregressive models (e.g., GPT-4, LLaMA), the model predicts **one token at a time**.  
- **Multi-token prediction** generates **multiple tokens per step**, reducing inference latency.  

### **Why is it useful?**  
- Reduces sequential dependency in autoregressive models.  
- Helps in speeding up **training and inference**.  
- Often used with **speculative decoding** (guessing multiple tokens and validating them).  

✅ **Used In:** GPT-4 Turbo, Gemini, Mistral  



## **2️⃣ Auxiliary Loss-Free Strategy**  
### **What is it?**  
- Many models use **auxiliary losses** (e.g., contrastive loss, embedding regularization) during training.  
- **Auxiliary Loss-Free Strategy** removes these extra losses, making training **simpler** and potentially faster.  

### **Why is it useful?**  
- Reduces computational overhead.  
- Improves efficiency by focusing only on primary loss (e.g., cross-entropy for next-token prediction).  

✅ **Used In:** Optimized LLMs (GPT-4 Turbo, Llama 3)  



## **3️⃣ FP8 Mixed Precision Training**  
### **What is it?**  
- FP8 (Float 8-bit) is an **ultra-low precision format** for training large models.  
- Traditional models use **FP16** or **BF16**, but FP8 **further reduces memory usage**.  

### **Why is it useful?**  
- Reduces memory footprint, allowing **larger models** to fit on **smaller GPUs**.  
- **Improves speed** by reducing the amount of data transferred between compute units.  
- **Helps scale models beyond 1T parameters**.  

✅ **Used In:** NVIDIA Hopper GPUs, Gemini 1.5, GPT-4 Turbo  



## **4️⃣ Computation-to-Communication Ratio & Overlaps**  
### **What is it?**  
- Measures how much **computation** happens compared to **communication** (e.g., data transfer between GPUs/nodes).  
- **Higher ratio = better efficiency** (less waiting for data).  

### **Overlaps?**  
- **Overlapping computation with communication** means running some computation **while data is being transferred**, avoiding idle time.  
- Used in **distributed training** to keep GPUs **fully utilized**.  

✅ **Used In:** High-performance distributed training (GPT-4, Gemini, DeepSpeed, Megatron-LM)  


## **5️⃣ Cross-Node All-to-All Communication Kernels**  
### **What is it?**  
- In **distributed training**, multiple GPUs/nodes **exchange model parameters** to stay in sync.  
- **All-to-All communication** means every node **shares** its updates with every other node.  

### **Why is it useful?**  
- Balances workload across **thousands of GPUs**.  
- Used in **Mixture of Experts (MoE)** models, where different experts need to communicate efficiently.  

✅ **Used In:** GPT-4 Turbo, Gemini 1.5, LLaMA 3  


## **6️⃣ Autoregressive Models**  
### **What are they?**  
- **Autoregressive models generate text token-by-token**, predicting each token based on previous ones.  
- **Examples:** GPT-4, LLaMA, Mistral.  

### **Pros & Cons**  
✅ **Pros:**  
- **High accuracy** (each token depends on all previous tokens).  
- **Easy to train with teacher forcing (cross-entropy loss).**  

❌ **Cons:**  
- **Slow inference** (must generate tokens **sequentially**).  
- Needs **KV caching** to speed up decoding.  

✅ **Optimized with:** FlashAttention, Speculative Decoding, Multi-Token Prediction  



## **7️⃣ Rotary Positional Embeddings (RoPE)**  
### **What is it?**  
- A **positional encoding technique** that enables **better handling of long context lengths**.  
- Instead of adding a fixed position embedding, RoPE **rotates token embeddings in multi-dimensional space**.  

### **Why is it useful?**  
- Supports **longer context lengths (128K+ tokens)** without degrading quality.  
- Allows for **extrapolation** to longer sequences.  



✅ **Used In:** LLaMA, GPT-4, Claude 3, Gemini  



## **🔥 Final Thoughts**  
| **Concept** | **Purpose** | **Used In** |
|-------------|------------|-------------|
| **Multi-Token Prediction** | Faster inference | GPT-4 Turbo, Gemini |
| **Auxiliary Loss-Free Strategy** | Efficient training | LLaMA 3, GPT-4 Turbo |
| **FP8 Mixed Precision** | Reduce memory, faster training | NVIDIA Hopper GPUs, Gemini |
| **Computation-to-Communication Ratio** | Optimize multi-GPU training | DeepSpeed, Megatron-LM |
| **Cross-Node All-to-All Kernels** | Synchronize large models | GPT-4, Gemini 1.5 |
| **Autoregressive Models** | Generate text step-by-step | GPT-4, Claude, Mistral |
| **Rotary Positional Embeddings (RoPE)** | Handle long context | LLaMA, GPT-4, Gemini |

These optimizations **enable modern LLMs to scale efficiently**, **process longer contexts**, and **generate text faster**.

----
---

### **1️⃣ Fine-Grained Quantization**  
Fine-grained quantization applies different bit-widths to specific layers, weights, or activations in a model to optimize memory, computational efficiency, and accuracy, balancing performance with reduced precision.  

### **2️⃣ Wgrad Operation**  
Wgrad (Weight Gradient Computation) is the process of calculating the gradient of the loss function with respect to model weights during backpropagation, which is computationally intensive and optimized using mixed precision and communication overlap strategies.  

### **3️⃣ Prefilling Stage in LLMs**  
The prefilling stage in LLMs initializes the model with input tokens, embeddings, and cached attention states before generation, optimizing memory and computation for efficient inference or training.  

### **4️⃣ Ablation Studies (Single Line)**  
Ablation studies systematically remove or modify parts of a model to assess their impact, identifying the most critical components for performance.
---
---

## Web Scraping with Python
https://huggingface.co/spaces/HuggingFaceFW/blogpost-fineweb-v1

Suppose you have to pull large amount of data from websites and you want to fetch it as quickly as possible. How would you do it? Manually going to the website and collect those datas.It will be tedious work. So, "web scrapping" will help you out in this situation. Web scrapping just makes this job easier and faster.

Here, we will do Web Scrapping with Python, starts with

1. Why we do web scrapping?
Web scrapping will be used to collect large amount of data from Websites.But why does someone have to collect such large amount of data from websites? To know about this, let's have to look at the applications of web scrapping:

- __Price comaparison:__ Parsehub is providing such services to useweb scraping to collect data from some online shopping websites and use to comapre price of products from another.
- __Gathering Emails:__ There are lots of companies that use emails as a medium for marketing, they use web scrapping to collect email id's and send bulk emails.
- __Social media scrapping:__ Web scrapping is used to collect data from Social Media websites such as Twitter to find out what's trending in twitter.
- __Research and Development:__ For reasearch purposes people do web scrapping to collect a large set of data(Statistics, General information, temperature,etc.) from websites, which are analyzed and used to carry out surveys or for R&D.


2. What is Web scrapping and is it legal or not?
Web scrapping is an automated to extract large amount of data from websites.And the websites data are unstructured most of the time.Web scrapping will help you out to collect those unstructured data and stored it in a structured form.There are different ways to scrape websites such as online services,APIs, or by writing your own code. Here, we'll see how to implementing the web scraping with python.

Coming to the question, is scrapping legal or not? Some websites allow web scrapping and some not.To know whether website allows you to scrape it or not by website's "robots.txt" file. You can find this file just append "/robots.txt" to the URL that you want to scrape.Here, we're scrapping from Flipkart website.So, to see the "robots.txt" file, URL is www.flipkart.com/robots.txt. 

3. How does web scrapping work?

When we run the code for web scraping, a request is sent to the URL that you have mentioned in the code. As a response to the request, the server send the data and allows you to read the HTML or XML page. Then our code will parses the HTML or XML page, find the data and extract it.

To extract datas using web scraping with python, you need to follow these basic steps:

  1.Find that URL that you mentioned in the code and want to scrape it.
  2.Inspect the Page for scraping.
  3.Find those data you want to extract.
  4.Write the code for scrapping.
  5.Run the code and extract the data.
  6.Store the data in the required format.
  
Now lets see how to extract data from the flipkart website using Python.

4. Libraries used for Web scrapping

We already know, that python used for various applications and there are different libraries for different purposes.In this, we're using the following libraries:

- **Selenium:** Selenium library is used for web testing. We will use to automate browser activities.
- **BeautifulSoup4:** It is generally used for parsing HTML and XML documents.It creates a parse trees that is helpful to extract the datas easily.
- **Pandas:** It is a Python library used for data manipulation and analysis.Pandas is used to extract data and stored it in the desired format.

5. For Demo Purpose : Scrapping a Flipkart Website

Pre-requisites:

 - Python 3.x with Selenium, Beautifulsoup4, Pandas library  installed.
 - Google Chrome Browser
 
You can go through this [link](https://github.com/iNeuronai/webscrappper_text.git) for more details.

---
---

---
---

## **vocabulary in nlp**

Corpus: A large and structured set of texts used for linguistic analysis and research.  
Example: "The Brown Corpus is commonly used for NLP research."

Document: An individual piece of text within a corpus, ranging from a single sentence to an entire book.  
Example: "Each news article in a corpus is considered a separate document."

Vocabulary: The set of unique words present in a corpus or document.  
Example: "In the sentence 'I love NLP,' the vocabulary is {I, love, NLP}."

Term: A word or group of words that conveys a specific meaning within a particular context.  
Example: "In a machine learning document, terms like 'algorithm' and 'data' are important."

Token: An individual unit of text, such as a word or punctuation mark, obtained by breaking down a sentence during tokenization.  
Example: "'NLP is fun!' → ['NLP', 'is', 'fun', '!']."

Semantic Analysis: The process of understanding the meaning of words and sentences to derive insights from text.  
Example: "'The bank can refuse to lend money' differentiates 'bank' as a financial institution."

Pragmatics: The study of how context influences the interpretation of meaning in language.  
Example: "'Can you pass the salt?' is understood as a request."

Word Sense Disambiguation (WSD): The task of determining which meaning of a word is being used in a given context.  
Example: "'Bark' can refer to a dog's sound or a tree's outer layer."

Lexicon: A collection of words and their meanings, including usage and grammatical information.  
Example: "A thesaurus lists synonyms for words."

Topic Modeling: A technique to identify underlying topics in a collection of documents.  
Example: "Topic modeling reveals that documents about 'sports' frequently discuss 'teams' and 'players.'"

Dependency Parsing: A syntactic analysis that identifies relationships between words in a sentence.  
Example: "'The cat sat on the mat' shows 'sat' as the main verb, 'cat' as the subject."

Text Classification: Assigning predefined categories or labels to text based on its content.  
Example: "A review 'This product is great!' could be classified as positive sentiment."

Ensemble Learning: A technique that combines multiple models to improve overall performance.  
Example: "Using both a decision tree and a logistic regression model for email classification."

---
https://youtu.be/ENLEjGozrio?si=94jkx5ZiBvN0HVAc

---


## Data Preprocessing Steps in NLP


| **Text Cleaning Step**         | **Example**                                   | **spaCy Function Used**                         |
|---------------------------------|-----------------------------------------------|------------------------------------------------|
| **Text Cleaning**               | "Hello, World! 123" → "hello world"          | `spacy.lang.en.stop_words` (for stopwords removal), custom regex patterns for noise removal |
| **Lowercasing**                 | "This is a Sample Text" → "this is a sample text" | No direct function, use `str.lower()` on the text or `spacy.tokens.Token.lower_` for token-level lowercase |
| **Removing Punctuation**        | "Hello, world!" → "Hello world"              | `spacy.tokens.Token.is_punct` for identifying and removing punctuation tokens |
| **Removing Numbers**            | "I have 2 apples" → "I have apples"          | `spacy.tokens.Token.is_digit` for identifying and removing digits |
| **Removing Whitespace**         | "  Hello   world!  " → "Hello world!"        | `str.strip()` to remove leading and trailing spaces, or `spacy.tokens.Token.is_space` for in-between spaces |
| **Text Normalization**          | "can't" → "cannot"                           | Use `spacy.matcher` for custom patterns or regular expressions for specific normalizations (e.g., expanding contractions) |



## REGEX-

| **Function**       | **Definition**                                                                                      |
|--------------------|-----------------------------------------------------------------------------------------------------|
| `.`                | Matches any character except a newline.                                                              |
| `^`                | Anchors the regex to the start of a string.                                                          |
| `$`                | Anchors the regex to the end of a string.                                                            |
| `\d`               | Matches any digit (equivalent to `[0-9]`).                                                           |
| `\D`               | Matches any non-digit character (equivalent to `[^0-9]`).                                           |
| `\w`               | Matches any word character (alphanumeric plus underscore: `[a-zA-Z0-9_]`).                          |
| `\W`               | Matches any non-word character (equivalent to `[^a-zA-Z0-9_]`).                                     |
| `\s`               | Matches any whitespace character (spaces, tabs, newlines).                                          |
| `\S`               | Matches any non-whitespace character.                                                                |
| `[]`               | Denotes a character class, matching any one of the characters inside the brackets.                   |
| `[^]`              | Denotes a negated character class, matching any character not inside the brackets.                   |
| ``                | Acts as an OR operator, matching either the pattern on the left or right.                           |
| `()`               | Groups expressions together.                                                                         |
| `?`                | Makes the preceding character or group optional (0 or 1 occurrence).                                 |
| `*`                | Matches the preceding character or group 0 or more times.                                           |
| `+`                | Matches the preceding character or group 1 or more times.                                           |
| `{n}`              | Matches exactly `n` occurrences of the preceding character or group.                               |
| `{n,}`             | Matches `n` or more occurrences of the preceding character or group.                               |
| `{n,m}`            | Matches between `n` and `m` occurrences of the preceding character or group.                       |
| `\b`               | Matches a word boundary (position between a word and a non-word character).                         |
| `\B`               | Matches a non-word boundary.                                                                         |
| `\`                | Escapes a special character, allowing it to be treated as a literal.                                 |
| `(?=...)`          | Positive lookahead: asserts that what follows matches the specified pattern.                         |
| `(?!...)`          | Negative lookahead: asserts that what follows does not match the specified pattern.                 |
| `(?<=...)`         | Positive lookbehind: asserts that what precedes matches the specified pattern.                      |
| `(?<!...)`         | Negative lookbehind: asserts that what precedes does not match the specified pattern.               |


---
---




## Filtering and Transformation



| **Task**                      | **Method**                                                                                      | **Condition/Explanation**                                                                 |
|-------------------------------|-------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| **Stop Words Removal**         | **Using `token.is_stop`**                                                                        | Removes stop words based on spaCy’s default stop words list.                             |
|                               | **Custom Stop Words List**                                                                       | When you want to filter out specific stop words that are not in the default list.         |
|                               | **Domain-Specific Stop Words**                                                                   | Removes stop words tailored to a particular domain or context (e.g., "software", "app").  |
| **Stemming**                   | **Using `token.lemma_` (spaCy’s lemmatization)**                                                  | When you need to reduce words to their base forms with correct context (preferred over stemming). |
|                               | **Using `PorterStemmer` or `SnowballStemmer` (NLTK)**                                             | If you need more aggressive stemming, stripping suffixes/prefixes.                       |
| **Lemmatization**              | **Using `token.lemma_`**                                                                          | Standard method for reducing words to their base form considering meaning and context.    |
|                               | **Using POS Tagging (`token.pos_`) for context-based lemmatization**                             | When you want to ensure lemmatization takes the part of speech (e.g., "better" → "good"). |
|                               | **Custom Lemmatizer Rules**                                                                      | When you need to define specific lemmatization rules for non-standard forms.             |


---
---

## Tokenization and Structuring

https://tiktokenizer.vercel.app/

| **Task**                        | **Method**                                                                                             | **Condition/Explanation**                                                                 |
|----------------------------------|--------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| **Tokenization**                 | **Using spaCy's `nlp()` pipeline**                                                                    | Standard tokenization into words or sentences using spaCy's default pipeline.            |
|                                  | **Sentence-level Tokenization**                                                                        | Tokenization can also be done at the sentence level by using `doc.sents` for sentence segmentation. |
|                                  | **Custom Tokenizer**                                                                                  | When you need to define specific tokenization rules, e.g., handling punctuation or special characters differently. |
| **Part-of-Speech (POS) Tagging** | **Using `token.pos_`**                                                                                 | Assigning POS tags to each token using spaCy’s default POS tagger.                       |
|                                  | **Using `token.dep_` for syntactic relationships**                                                     | Analyzing the syntactic structure along with POS tags to get more granular insights (e.g., subject, object). |
|                                  | **Custom POS Tagging**                                                                                 | When you want to train or use custom models for specific POS tagging for domain-specific tasks. |
| **Named Entity Recognition (NER)** | **Using `spacy.ner` in the default pipeline**                                                          | Identifying named entities using spaCy’s pre-trained NER model.                         |
|                                  | **Custom NER Model**                                                                                   | When you need to train a custom NER model for specific entities, e.g., domain-specific names or categories. |
|                                  | **Entity-based Classification**                                                                         | Identifying named entities and classifying them into custom categories beyond the standard ones (e.g., “COMPANY”, “PRODUCT”). |



---
---

Ah, I see what you're asking now! You're looking for a breakdown of **different methods** of **Word Frequency Analysis** (like TF-IDF, Word2Vec, etc.), not just how we execute them.

Here's the revised table, where I focus on the **methods** of word frequency analysis, with explanations of each:

---

## **Analysis and Representation**

| **Technique**                      | **Description**                                                                                 | **Example**                                              | **Methods for Analysis**                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|-------------------------------------|-------------------------------------------------------------------------------------------------|----------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Word Frequency Analysis**         | Analyzing the frequency of words to identify common terms and insights within the dataset.      | In "apple apple banana", the word frequency is {"apple": 2, "banana": 1}. | - **Raw Frequency Count**: Count occurrences of each word, ignoring context (simple count of each word in the corpus).<br> - **TF-IDF (Term Frequency-Inverse Document Frequency)**: A statistical measure that evaluates the importance of a word in a document relative to the corpus.<br> - **Word2Vec**: A model that converts words into vector representations, capturing contextual meaning, often used for semantic analysis.<br> - **GloVe (Global Vectors for Word Representation)**: Similar to Word2Vec, but generates word vectors by leveraging global word-word co-occurrence statistics from the corpus. |
| **N-grams Analysis**               | Generating continuous sequences of n items (words or characters) from a given text.             | For the sentence "I love NLP", bigrams are ["I love", "love NLP"]. | - **Unigrams**: Single word tokens.<br> - **Bigrams**: Sequences of two adjacent words.<br> - **Trigrams**: Sequences of three adjacent words.<br> - **Skip-grams**: A Word2Vec method where the goal is to predict the context of a word, typically used for n-gram generation in larger corpora.                                                                                                                                     |
| **Bag of Words (BoW)**             | A representation of text that describes the occurrence of words within a document.              | "I love apples" becomes [1, 1, 1, 0, 0] for ["I", "love", "apples", "bananas", "oranges"]. | - **Simple BoW**: A frequency-based model that represents words as a sparse vector based on word occurrence.<br> - **TF-IDF BoW**: Uses the TF-IDF weighting instead of raw word counts to account for the importance of words across the entire corpus.<br> - **Word2Vec BoW**: Words are represented as vectors, with contextual relations captured.<br> - **GloVe BoW**: Word vectors generated by GloVe can also be used in a similar way to BoW to represent text. |
| **Personal Frequency Distribution** | Analyzing how frequently specific terms or entities appear, often useful for personalized recommendations. | Analyzing how often user-specific terms (e.g., product names) appear. | - **Named Entity Recognition (NER)**: Extracts entities (e.g., products, places, people) and counts their occurrences using tools like `spaCy`.<br> - **Collaborative Filtering**: Analyzes item (or user) interactions based on frequency of mentions across users (e.g., recommendation systems).<br> - **Topic Modeling**: Identifies topics from text and tracks the frequency of topics in user profiles. |

---




## Embedding Techniques






## Frequency-Based Embedding

 1. **Count Vectors**
- **Definition**: Count vectors represent text data as a matrix where each element is the count of a term in the document. Each unique word in the corpus becomes a dimension in the vector space.
- **Example**: Given two sentences:
  - Sentence 1: "I love NLP"
  - Sentence 2: "I enjoy NLP"
  
  The count vector representation might look like this:
  
| Word | Count in Sentence 1 | Count in Sentence 2 |
|------|----------------------|----------------------|
| I    | 1                    | 1                    |
| love | 1                    | 0                    |
| enjoy| 0                    | 1                    |
| NLP  | 1                    | 1                    |

2. **TF-IDF (Term Frequency-Inverse Document Frequency)**
- **Definition**: TF-IDF is a numerical statistic that reflects the importance of a word in a document relative to a collection of documents (corpus). It increases proportionally to the number of times a word appears in the document and is offset by the frequency of the word in the corpus.
- **Formula**:
  
  $$ \text{TF-IDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t) $$

  Where:
  
  - $ \text{TF}(t, d) = \frac{\text{Number of times term t appears in document d}}{\text{Total number of terms in document d}} $
  
  - $ \text{IDF}(t) = \log\left(\frac{\text{Total number of documents}}{\text{Number of documents containing term t}}\right) $
  
- **Example**: If "NLP" appears 3 times in a document of 100 words and appears in 5 out of 100 documents:
  
  $$ \text{TF}(NLP) = \frac{3}{100} = 0.03 $$
  
  $$ \text{IDF}(NLP) = \log\left(\frac{100}{5}\right) = \log(20) \approx 1.301 $$
  
  Thus, \( \text{TF-IDF}(NLP) \approx 0.03 \times 1.301 \approx 0.03903 \).

3. **Co-Occurrence Matrix**
- **Definition**: A co-occurrence matrix counts how often pairs of words occur together in a given context (e.g., within the same sentence or window). This matrix helps to identify relationships and associations between words.
- **Example**: For the sentences "I love NLP" and "I enjoy NLP", the co-occurrence matrix may look like this:

|     | I | love | enjoy | NLP |
|-----|---|------|-------|-----|
| I   | 2 | 1    | 1     | 2   |
| love| 1 | 0    | 0     | 1   |
| enjoy| 1| 0    | 0     | 1   |
| NLP | 2 | 1    | 1     | 0   |

## Prediction-Based Embedding

 1. **Word2Vec**
- **Definition**: Word2Vec is a prediction-based model that generates word embeddings by predicting words in a given context (CBOW) or predicting context words given a target word (Skip-gram).

 a. **CBOW (Continuous Bag of Words)**
- **Definition**: In CBOW, the model predicts the target word based on the context words surrounding it.
- **Example**: Given the context words "the", "cat", and "sat", CBOW would predict the target word "on".

 b. **Skip-Gram**
- **Definition**: In the Skip-gram model, the model predicts the context words given a target word.
- **Example**: Given the target word "on", Skip-gram would predict the surrounding context words "the", "cat", and "sat".

 2. **GloVe (Global Vectors for Word Representation)**
- **Definition**: GloVe is a global vector representation model that utilizes aggregated global word-word co-occurrence statistics from a corpus to generate embeddings. It aims to derive the embedding such that the dot product of two word vectors predicts their co-occurrence probability.
- **Formula**:
  
  $$ P_{ij} = \frac{X_{ij}}{X_j} $$

  Where:
  
  - \( P_{ij} \) is the probability of word \( i \) occurring in the context of word \( j \).
  - \( X_{ij} \) is the co-occurrence count of word \( i \) and word \( j \).
  
- **Example**: For the co-occurrence matrix generated from a corpus, GloVe learns embeddings by factorizing the matrix to obtain word vectors that represent words based on their global context.

---
---
## compare two strings

### 1. **Exact Match**

   - Simply use the equality operator `==` in most programming languages (e.g., `string1 == string2`). This checks if both strings are identical, character by character.

### 2. **Case-Insensitive Comparison**

   - Convert both strings to lowercase (or uppercase) and then compare. This method helps when case differences are irrelevant.

     ```python
     string1.lower() == string2.lower()
     ```

### 3. **Levenshtein Distance (Edit Distance)**

   - The **Levenshtein Distance** measures the minimum number of single-character edits (insertions, deletions, or substitutions) required to change one string into the other. A smaller distance indicates more similarity.
   - This can be computed using libraries like `python-Levenshtein` or `editdistance` in Python.

     ```python
     import Levenshtein
     similarity = Levenshtein.distance(string1, string2)
     ```

### 4. **Jaccard Similarity**

   - Treat each string as a set of characters or words and calculate the Jaccard similarity, which is the ratio of the intersection of the sets to the union.

     $$
     \text{Jaccard Similarity} = \frac{|A \cap B|}{|A \cup B|}
     $$

   - For example, for `string1 = "apple"` and `string2 = "appeal"`, the Jaccard similarity would focus on common and unique characters.

### 5. **Cosine Similarity (TF-IDF)**

   - For longer strings, convert each string into a vector (such as TF-IDF) and calculate the **cosine similarity**, which measures the cosine of the angle between two vectors. Values close to 1 indicate higher similarity.
   - This approach is commonly used in text mining for comparing larger bodies of text.

     ```python
     from sklearn.feature_extraction.text import TfidfVectorizer
     from sklearn.metrics.pairwise import cosine_similarity

     vectorizer = TfidfVectorizer().fit_transform([string1, string2])
     similarity = cosine_similarity(vectorizer[0:1], vectorizer[1:2])
     ```

### 6. **N-gram Similarity**

   - Break each string into consecutive `n`-length character sequences (n-grams), then compare them for overlap. This method captures partial matches effectively and is useful in detecting similar substrings.

### 7. **Fuzzy Matching (Token Set or Token Sort)**

   - Libraries like `fuzzywuzzy` in Python provide various fuzzy matching techniques, such as `fuzz.ratio()` or `fuzz.partial_ratio()`, which are based on Levenshtein distance but offer more flexibility for approximate matches.

     ```python
     from fuzzywuzzy import fuzz

     similarity = fuzz.ratio(string1, string2)
     ```

### 8. **Jaro-Winkler Distance**

   - This distance metric accounts for transpositions and is particularly useful for short strings with small typographical errors. It gives higher weights to matches that start similarly, making it effective for name matching.

### Choosing the Right Method

The method you choose depends on the context:
- **Exact matches** or **case-insensitive checks** work well for strict comparisons.
- **Levenshtein**, **Jaccard**, or **Cosine similarity** work well for partial matches or text comparisons with minor errors.
- **Fuzzy matching** methods (like those in `fuzzywuzzy`) are helpful when comparing names, addresses, or strings with possible typographical errors.

---
---
Here’s the information about sampling novel sequences in the context of NLP:

**Sampling Novel Sequences:** A method used in generating new text or sequences by selecting from a probability distribution over possible next items (words, characters, etc.) based on a trained model. This approach aims to create diverse outputs rather than repeating known sequences, often leveraging randomness or temperature settings to control creativity.

**Example:** Given the model output probabilities for the next word after "The cat is," sampling might yield:

1. "The cat is sitting on the mat." (common continuation)
2. "The cat is a magician." (novel but plausible)
3. "The cat is flying to the moon!" (unexpected and creative)

By adjusting the sampling method (like greedy sampling, temperature sampling, or top-k sampling), the generated text can range from conservative to highly creative.

---
---
---

 ## 1. **Performance Metrics**:
   - **Perplexity**: Measures how well a probability model predicts a sample. Lower perplexity indicates better performance.
   - **BLEU Score**: Used for evaluating machine translation, comparing the model output with one or more reference translations. Higher scores indicate better performance.
   - **ROUGE Score**: Commonly used for summarization tasks, it measures the overlap between the model-generated output and reference summaries. Higher scores indicate better performance.
   - **Accuracy**: For classification tasks, the proportion of correct predictions.
   - **F1 Score**: The harmonic mean of precision and recall, especially useful for imbalanced datasets.

## 2. **Context Length Handling**:
   - **Maximum Context Length**: Evaluate how much longer sequences the new model can handle compared to the previous model. 
   - **Performance on Long Contexts**: Assess the performance metrics (e.g., perplexity, BLEU, ROUGE) specifically on longer input sequences to see if the new model maintains or improves performance as context length increases.

## 3. **Computational Efficiency**:
   - **Training Time**: Measure the time taken to train the model. A more efficient model can handle longer contexts without significantly increasing training time.
   - **Inference Time**: Measure how quickly the model generates outputs for given input lengths.

## 4. **Generalization Ability**:
   - **Evaluation on Diverse Datasets**: Test the new model on a wide range of datasets, especially those that include longer sequences. Improved performance across various domains indicates better generalization.

## 5. **Qualitative Evaluation**:
   - **Human Evaluation**: Gather human judgments on the quality of the outputs generated by the models, especially for tasks like text generation, translation, or summarization. Assess aspects such as coherence, relevance, and fluency.

## 6. **Robustness**:
   - **Stress Testing**: Evaluate how well the model performs under various input conditions, including noisy, incomplete, or misleading inputs. Check if the model can still provide meaningful outputs with longer contexts.

### Example Comparison

| Criteria                     | Previous Model               | New Model                    |
|------------------------------|------------------------------|------------------------------|
| Maximum Context Length        | 512 tokens                   | 2048 tokens                  |
| Perplexity (on validation set)| 35.0                         | 30.5                         |
| BLEU Score (on translation task)| 25.2                      | 28.7                         |
| Training Time                 | 10 hours                     | 12 hours                     |
| F1 Score (for classification) | 0.85                       | 0.88                         |
| Human Evaluation (score out of 5) | 3.5                     | 4.2                          |

 

---
---

Errors from pretraining model from mistral 7b model on recipie dataset:

# Comprehensive Table of All Errors and Solutions

| **Error Message/Type** | **Possible Cause** | **Solution/Fix** |
|------------------------|-------------------|-------------------|
| **Tokenizer AttributeError** | Tried to access tokenizer from `MistralForCausalLM` | Load tokenizer separately |
| **Padding Token Missing** | Mistral tokenizer lacks a default padding token | Set `tokenizer.pad_token = tokenizer.eos_token` |
| **Dataset Format Error** | `SFTTrainer` got an incorrect dataset format | Ensure dataset is loaded with `load_from_disk()` and correct splits are passed |
| **ChatML Applied Twice** | `SFTTrainer` tried formatting again | Remove non-tokenized columns (`"text"`, `"question"`, `"answer"`) |
| **Incorrect Dataset Format in `SFTTrainer`** | Passed full dataset instead of splits | Use `train_dataset=tokenized_dataset["train"]` |
| **Colab Freezing** | High RAM/GPU usage | Run `!nvidia-smi`, restart runtime if needed |
| **Saving Model Issues** | Model needed to be saved in both locations | Use `trainer.save_model()` for both disk and Drive |
| **"No such file or directory: 'workspace_utils.py'"** | Missing file in project directory | Ensure `workspace_utils.py` is present or update script to not depend on it |
| **"Replace with your actual padding token index"** | Placeholder not replaced in script | Set actual padding token index: `tokenizer.pad_token_id = tokenizer.eos_token_id` |
| **"TypeError: Population must be a sequence"** | `random.sample()` used on dict instead of list | Convert dataset split to list: `eval_samples = random.sample(list(dataset["test"]), min(30, len(dataset["test"])))` |
| **"CUDA out of memory"** | Model/batch size too large for available GPU memory | Reduce batch size, use mixed precision (`fp16=True`), or move to CPU if needed |
| **"size mismatch for lora layers"** | Incorrect LoRA adapter weights applied to base model | Ensure LoRA adapters are trained for same model version and correctly loaded |
| **"Can I use this model without Mistral?"** | LoRA adapters require base model | No, because LoRA only saves *delta weights*. You need Mistral as the base model |
| **`SyntaxError: invalid syntax`** in `ast.literal_eval(x)` | `ast.literal_eval()` failed due to unescaped characters | Use `json.loads()` instead, replacing single quotes with double quotes for valid JSON parsing |
| **`'Series' object has no attribute 'applymap'`** | `applymap()` only works on DataFrames, not Series | Use `apply()` instead of `applymap()` for Series processing |
| **`'DataFrame' object has no attribute 'compute'`** | cuDF DataFrame doesn't have `.compute()` (used in Dask) | Remove `.compute()` and ensure all cuDF operations are done within cuDF itself |
| **`AttributeError: 'Series' object has no attribute 'to_pandas'`** | `series.to_pandas()` is not a valid cuDF operation | Use `series.to_pandas()` only when converting back to pandas |
| **`Error processing batch: memory allocation failed`** | GPU ran out of memory due to large batch sizes | Reduce `gpu_batch_size` and add `cp.cuda.Device(0).synchronize()` to clear memory |
| **`json.decoder.JSONDecodeError: Expecting value`** | Empty or malformed JSON string | Use `str.replace("'", '"')` before parsing JSON |
| **`ValueError: Columns must be same length as key`** | Mismatched DataFrame assignments after `apply()` | Ensure `safe_list_eval_gpu` returns correctly sized `cudf.Series` |
| **`TypeError: 'NoneType' object is not iterable`** | `NER` column had `None` values, breaking `.join()` operations | Replace `None` with empty lists using `.fillna("[]")` |
| **OSError: You are trying to access a gated repo** | The model requires special access on Hugging Face | Request access on the HF model page and log in using `huggingface-cli login` |
| **OSError: We couldn't connect to 'https://huggingface.co'** | Internet connection issue or incorrect model path | Check internet connection and verify the model name is correct |
| **ImportError: FlashAttention2 has been toggled on...** | `flash_attn` is not installed | Install it using `pip install flash-attn` |
| **ValueError: No columns in the dataset match the model's forward method** | Dataset format doesn't match model expectations | Set `remove_unused_columns=False` in `TrainingArguments` |
| **ValueError: n is not supported, only azure_ml...** | `report_to="none"` is not a valid option | Use `report_to=[]` instead of `"none"` to disable logging |
| **Ran out of disk space** | Model download and caching used too much storage | Free up disk space or store the model in Google Drive |
| **Ran out of RAM (Runtime Crashed)** | Model too large for available RAM | Use quantization (`load_in_4bit=True`), reduce batch size, or upgrade runtime |
| **KeyError: 'input_ids'** | Tokenized dataset missing `input_ids` | Check dataset structure using `print(dataset["train"].column_names)` |
| **KeyError: 'question'** | Dataset missing expected column | Verify dataset columns and rename if necessary |
| **Nothing was printed (No Output)** | Script not running or dataset issues | Add verification prints, check paths with `os.path.exists()` |
| **❌ Dataset path does not exist!** | Missing or incorrectly located dataset folder | Verify correct path with `os.path.abspath()` |
| **❌ Error loading dataset** | Corrupted or improperly saved dataset | Reload dataset with `load_from_disk()` or reprocess if corrupted |
| **Colab Crashing During Tokenization** | Dataset too large for memory | Use `num_proc=4` in `.map()` to enable multiprocessing |
| **FileNotFoundError: No such files: '.../dataset_info.json'** | Incorrect dataset path in `load_from_disk()` | Use root directory path instead of `train/` subdirectory |
| **cp: cannot overwrite non-directory with directory** | Path conflict during copying | Remove existing file/folder with `!rm -rf` before copying again |
| **Slow dataset loading from Google Drive** | I/O limits when accessing Drive directly | Copy dataset to Colab's local disk before loading |
| **DataLoader Error: Expected tensor, got list** | Incorrectly formatted tokenized dataset | Ensure `collate_fn=data_collator` is used when creating `DataLoader` |

- what is lora finetuning?
- what are the sftttuner function takes as input...does it need only text..
- 4bit vs 8bit loading of base model
- can't we load pretrained model separately when finetuned with lora

Based on the document, here are 5 key insights for each of the main topics discussed:

### Fine-Tuning Approaches for Question-Answering Models
1. Few-shot fine-tuning with prompting avoids creating massive datasets by using examples in the prompt to establish patterns
2. Reference-based query generation retrieves information from structured data rather than generating answers from scratch
3. Using a small knowledge base with Q&A templates allows matching user questions to relevant templates
4. Two-phase fine-tuning (raw text followed by Q&A pairs) builds both general language understanding and specific question-answering capabilities
5. Incremental training can update models with new knowledge without full retraining

### Model Optimization Techniques
1. Loading models in 4-bit precision (load_in_4bit=True) drastically reduces VRAM usage compared to 16-bit precision
2. Using device_map="auto" distributes model layers across available devices (GPU & CPU) to prevent memory errors
3. Avoiding explicit model.to(device) operations prevents moving the entire model to GPU at once
4. Quantization reduces model size while maintaining acceptable performance
5. These optimizations can reduce memory requirements from >30GB to <15GB VRAM

### LoRA (Low-Rank Adaptation) Benefits
1. Allows fine-tuning of quantized models by training only small parts
2. Significantly reduces memory usage by limiting trainable parameters
3. Helps avoid CUDA out-of-memory errors during training
4. Works effectively on GPUs with less than 16GB VRAM
5. Maintains model performance while enabling training on limited hardware

### Dynamic Model Updating
1. Logging missed recipes helps identify knowledge gaps in the model
2. Manual dataset updates can address specific knowledge gaps
3. Periodic retraining with updated datasets improves model knowledge
4. External knowledge integration can pull in new data without frequent fine-tuning
5. Incremental training allows updating models with new knowledge without full retraining

### Performance Comparisons
1. Traditional loading methods require >30GB VRAM and can take >10 minutes
2. Optimized loading with 4-bit quantization uses <15GB VRAM and takes ~5 minutes
3. Full-precision models cause out-of-memory issues on consumer GPUs (~15GB VRAM)
4. Optimized loading is approximately 5 times faster than traditional methods
5. Quantization and automatic device mapping are key to efficient model deployment